In [2]:
%matplotlib qt
from matplotlib import pyplot as plt
from numpy.random import rand
import skimage as sm
from skimage.io import imread, imsave
from skimage.util import img_as_float, random_noise
from skimage.transform import rotate
from skimage.filters import gaussian as ski_gaussian
from pylab import ginput
from scipy.signal import convolve, convolve2d, correlate2d, fftconvolve
from scipy.signal import  gaussian as scipy_gaussian
from scipy.fft import fft2, fftshift, ifft2, ifftshift
import numpy as np
import os
import timeit

os.chdir("../Mats")

In [3]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# 2

2.1

In [128]:
def apply_filter(image, filter):
    cop_image = np.copy(image)
    out_image = np.zeros(image.shape)
    row = (filter.shape[0]-1)//2
    column = (filter.shape[1]-1)//2
    for i in range(cop_image.shape[0]-filter.shape[0]):
        for j in range(cop_image.shape[1]-filter.shape[1]):
            out_image[i+row,j+column] = np.sum(cop_image[i:i+filter.shape[0],j:j+filter.shape[1]]*filter)
    return out_image


def fft_convolve(image, filter):
    row = (filter.shape[0]-1)//2
    column = (filter.shape[1]-1)//2
    cop_image = np.copy(image)
    padded_filter = np.pad(filter, ((0, (cop_image.shape[0]-filter.shape[0])), (0, (cop_image.shape[1]-filter.shape[1]))), 'constant')
    padded_filter = np.roll(padded_filter, cop_image.shape[0]-filter.shape[0]//2, axis=0)
    padded_filter = np.roll(padded_filter, cop_image.shape[1]-filter.shape[1]//2, axis=1)
    print(padded_filter[0,0])
    return np.real(ifft2(fft2(cop_image)*fft2(padded_filter)))

def mean_kernel(N):
    return np.ones((N, N))/N**2

A = imread("cameraman.tif")
kernels = [mean_kernel(5), mean_kernel(9)]
print(type(fft2(A)))

img_reg_3 = fft_convolve(A, kernels[1])
img_fft_3 = fftconvolve(A, kernels[1], mode="same")
img_reg_7 = apply_filter(A, kernels[1])
img_fft_7 = convolve2d(A, kernels[1], mode="same")
diff_3 = np.abs(img_fft_3-img_reg_3)
diff_7 = np.abs(img_fft_7-img_reg_7)
print(img_reg_3)
reg_kernel_time = []
fft_kernel_time = []

""" for i in range(2, 16):
    kernel = mean_kernel(i)
    reg_kernel_time.append(timeit.timeit(lambda: apply_filter(A, kernel), number=10))
    fft_kernel_time.append(timeit.timeit(lambda: fft_convolve(A, kernel), number=10))
    print(i) """


<class 'numpy.ndarray'>
0.012345679012345678
[[119.72839506 121.55555556 122.43209877 ... 117.59259259 119.24691358
  118.45679012]
 [122.82716049 124.87654321 125.88888889 ... 121.5308642  122.88888889
  122.07407407]
 [126.45679012 128.39506173 129.83950617 ... 124.82716049 126.28395062
  125.58024691]
 ...
 [115.49382716 116.67901235 117.2962963  ... 114.58024691 115.75308642
  115.13580247]
 [116.22222222 117.38271605 118.17283951 ... 115.43209877 116.69135802
  116.        ]
 [117.12345679 118.61728395 119.7037037  ... 114.85185185 116.48148148
  115.9382716 ]]


' for i in range(2, 16):\n    kernel = mean_kernel(i)\n    reg_kernel_time.append(timeit.timeit(lambda: apply_filter(A, kernel), number=10))\n    fft_kernel_time.append(timeit.timeit(lambda: fft_convolve(A, kernel), number=10))\n    print(i) '

In [ ]:
#jakob implementation
def apply_filter(image, filter):
    cop_image = np.copy(image)
    out_image = np.zeros(image.shape)
    row = (filter.shape[0]-1)//2
    column = (filter.shape[1]-1)//2
    for i in range(cop_image.shape[0]-filter.shape[0]):
        for j in range(cop_image.shape[1]-filter.shape[1]):
            out_image[i+row,j+column] = np.sum(cop_image[i:i+filter.shape[0],j:j+filter.shape[1]]*filter)
    return out_image


def fft_convolve(image, filter):
    cop_image = np.copy(image)
    prod = fft2(image, image.shape) * fft2(filter, image.shape)
    ifftprod = ifft2(prod).real
    convImg = np.roll(ifftprod, (-((filter.shape[0] - 1)//2),
                                 -((filter.shape[1] - 1)//2)), axis=(0, 1))
    return convImg


def mean_kernel(N):
    return np.ones((N, N))/N**2

A = imread("cameraman.tif")
kernels = [mean_kernel(5), mean_kernel(7)]
print(A.shape)

img_reg_3 = fft_convolve(A, kernels[1])
img_fft_3 = convolve2d(A, kernels[1], boundary="wrap",mode="same") #Changed this to be a wrap boundary, it still does fft now just cyclic which matches our implementeation
img_reg_7 = apply_filter(A, kernels[1])
img_fft_7 = convolve2d(A, kernels[1], mode="same")
diff_3 = np.abs(img_fft_3-img_reg_3)
diff_7 = np.abs(img_fft_7-img_reg_7)

reg_kernel_time = []
fft_kernel_time = []

""" for i in range(2, 16):
    kernel = mean_kernel(i)
    reg_kernel_time.append(timeit.timeit(lambda: apply_filter(A, kernel), number=10))
    fft_kernel_time.append(timeit.timeit(lambda: fft_convolve(A, kernel), number=10))
    print(i) """


(256, 256)


NameError: name 'fft' is not defined

In [ ]:
# np.save("reg_kernel_time.npy", np.array(reg_kernel_time))
# np.save("fft_kernel_time.npy", np.array(fft_kernel_time))
reg_kernel_time = np.load("reg_kernel_time.npy")
fft_kernel_time = np.load("fft_kernel_time.npy")

In [9]:
fig, ax = plt.subplots()
x_axis = np.arange(2, 16)
ax.plot(x_axis, reg_kernel_time, label="reg")
ax.plot(x_axis, fft_kernel_time, label="fft")
ax.set_xlabel("N"), ax.set_ylabel("Execution time (s)"), ax.set_title("Execution time for 100 executions for a filter with kernel NxN")
plt.tight_layout()
plt.legend()
plt.show()

In [129]:
fig, ax = plt.subplots(3, 2)
ax[0, 0].imshow(img_reg_3, cmap="gray"), ax[0,0].set_title("own fft"), ax[0, 0].axis("off")
ax[0, 1].imshow(img_reg_7, cmap="gray"), ax[0,1].set_title("own nested"), ax[0, 1].axis("off")
ax[1, 0].imshow(img_fft_3, cmap="gray"), ax[1,0].set_title("scipy fft"), ax[1, 0].axis("off")
ax[1, 1].imshow(img_fft_7, cmap="gray"), ax[1,1].set_title("scipy convolve"), ax[1, 1].axis("off")
ax[2, 0].imshow(diff_3, vmin=0, vmax=255, cmap="gray"), ax[2,0].set_title("Difference, fft"), ax[2, 0].axis("off")
ax[2, 1].imshow(diff_7, vmin=0, vmax=255, cmap="gray"), ax[2,1].set_title("Difference, convolve"), ax[2, 1].axis("off")
plt.tight_layout()
plt.show()

In [21]:
plt.imshow(A, cmap="gray")

In [74]:
kernel = mean_kernel(3)
img = []
img_reg_3 = apply_filter(A, kernels[0])
img_fft_3 = fft_convolve(A, kernels[0])
img_reg_7 = apply_filter(A, kernels[1])
img_fft_7 = fft_convolve(A, kernels[1])
diff_3 = np.abs(img_fft_3-img_reg_3)
diff_7 = np.abs(img_fft_7-img_reg_7)

reg_kernel_time = []
fft_kernel_time = []

for i in range(2, 16):
    kernel = mean_kernel(i)
    reg_kernel_time.append(timeit.timeit(lambda: apply_filter(A, kernel), number=100))
    fft_kernel_time.append(timeit.timeit(lambda: fft_convolve(A, kernel), number=100))
    print(i)

KeyboardInterrupt: 

In [58]:
A = imread("cameraman.tif")

kernels = [np.arange(1, 2**2+1).reshape((2,2)),np.arange(1, 3**2+1).reshape((3,3))]

img = convolve2d(A,kernels[0])
plt.subplot(2,2,1)
plt.imshow(img,cmap="gray")
img = convolve2d(A,kernels[1])
plt.subplot(2,2,2)
plt.imshow(img,cmap="gray")
img = fft_convolve(A,kernels[0])
plt.subplot(2,2,3)
plt.imshow(img,cmap="gray")
img = fft_convolve(A,kernels[1])
plt.subplot(2,2,4)
plt.imshow(img,cmap="gray")
plt.show()

In [ ]:
A = imread("cameraman.tif")

kernels = [np.arange(1, 2**2+1).reshape((2,2)),np.arange(1, 3**2+1).reshape((3,3))]

for i, kernel in enumerate(kernels):
    time = timeit.timeit(lambda: apply_filter(A, kernel), number=10)
    print(f"Nested For Loop: Image size {A.shape}, Kernel size {kernel.shape}, Execution time: {time/10} seconds")


images = [A,resize(A,(256,256*2)),resize(A,(256*2,256))]

# Measure execution times for FFT implementation
for i, image in enumerate(images):
    kernel = kernels[0]
    time = timeit.timeit(lambda: fft_convolve(image, kernel), number=10)
    print(f"FFT: Image size {image.shape}, Kernel size {kernel.shape}, Execution time: {time/10} seconds")

Nested For Loop: Image size (256, 256), Kernel size (2, 2), Execution time: 0.5441541300002427 seconds
Nested For Loop: Image size (256, 256), Kernel size (3, 3), Execution time: 0.5035541500001273 seconds
FFT: Image size (256, 256), Kernel size (2, 2), Execution time: 0.004148110000096494 seconds
FFT: Image size (256, 512), Kernel size (2, 2), Execution time: 0.00927501999976812 seconds
FFT: Image size (512, 256), Kernel size (2, 2), Execution time: 0.008910060000198427 seconds


In [ ]:


scipyA = fftconvolve(A,kernels[1])
ourA = fft_convolve(A,kernels[1])

plt.subplot(2,1,1)
plt.imshow(scipyA,cmap="gray")
plt.subplot(2,1,2)
plt.imshow(ourA,cmap="gray")

2.2

In [5]:
def waveAdd(image,a,v,w):
    x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    cos_wave = a * np.cos(v * x + w * y)
    img = image + cos_wave
    return img



def powerSpec(image):
    ft = fft2(np.copy(image))
    ftshift = fftshift(ft)
    ps = ftshift**2
    return np.abs(ps)

In [6]:
A = imread("cameraman.tif")
a, v, w = 50, 0.5, 0.5
B = waveAdd(A,a,v,w)

fftA = fft2(A)
fftA = fftshift(fftA)
fftB = fft2(B)
fftB = fftshift(fftB)
ps_A = powerSpec(A)
ps_B = powerSpec(B)

plt.subplot(2,2,1)
plt.imshow(A,cmap="gray"),plt.axis("off")
plt.title("Original")
plt.subplot(2,2,2)
plt.imshow(B,cmap="gray"),plt.axis("off")
plt.title("Original + noise")
plt.subplot(2,2,3)
plt.imshow(np.log10(1+ps_A),cmap="gray"),plt.xticks([], []),plt.yticks([], [])
plt.title("PS of original")
plt.subplot(2,2,4)
plt.imshow(np.log10(1+ps_B),cmap="gray"),plt.xticks([], []),plt.yticks([], [])
plt.title("PS + noise")
plt.show()

In [ ]:
#107, 107
#148, 148

In [245]:
def filterFunc(fft,pixel1,pixel2):
    N = fftB.shape[0]
    x,y = np.meshgrid(np.arange(N), np.arange(N))
    a1, a2 = 0.005, 0.005
    F1 = 1 - np.exp(-a1*(x-pixel1[0])**2-a2*(y-pixel1[1])**2)
    F2 = 1 - np.exp(-a1*(x-pixel2[0])**2-a2*(y-pixel2[1])**2)
    Z = F1*F2
    imgFs = fft*Z
    imgF = ifftshift(imgFs)
    imgF = ifft2(imgF)
    return imgF, Z

In [246]:
pixel1 = [107,107]
pixel2 = [148,148]
imgF, Z = filterFunc(fftB,pixel1,pixel2)

plt.subplot(2,2,1)
plt.imshow(B,cmap="gray"),plt.axis("off")
plt.title("Original + noise")
plt.subplot(2,2,2)
plt.imshow(np.real(imgF),cmap="gray"),plt.axis("off")
plt.title("Filtered image")
plt.subplot(2,2,3)
plt.imshow(np.log10(1+np.abs(fftB)),cmap="gray"),plt.xticks([], []),plt.yticks([], [])
plt.title("FFT + noise")
plt.subplot(2,2,4)
plt.imshow(Z,cmap="gray"),plt.xticks([], []),plt.yticks([], [])
plt.title("Filter")
plt.show()